In [1]:
#Practice with some ConvNets
#This ipython also assumes you have decent knowledge of ConvNets. This article should be good enough for now: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

In [2]:
#Let's import libraries 
import tensorflow as tf
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

In [3]:
#We'll start with MNIST first
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
"""(x_train, y_train), (x_test, y_test) tf.keras.datasets.cifar10.load_data()
In case you want to try CIFAR (NOTE: It would be good to check shape of Cifar and properly adjust since it is RGB)
"""

'(x_train, y_train), (x_test, y_test) tf.keras.datasets.cifar10.load_data()\nIn case you want to try CIFAR (NOTE: It would be good to check shape of Cifar and properly adjust since it is RGB)\n'

In [4]:
#Something to note is that CNN layers taken in channels as input as well. Since MNIST is grayscale, we just need to add singular channel dimension
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [5]:
#One-hot encoding
numpy_zero = np.zeros((y_train.shape[0], 10))
numpy_zero[np.arange(y_train.shape[0]), y_train] = 1
y_train = numpy_zero 
numpy_zero2 = np.zeros((y_test.shape[0], 10))
numpy_zero2[np.arange(y_test.shape[0]), y_test] = 1
y_test = numpy_zero2

In [6]:
#Don't forget to scale values
x_train = x_train / 255.0
x_test = x_test / 255.0 

In [7]:
#Ok so far we've just been doing the same things as our previous notebook. Let's actually get into the meat of ConvNets.
#Again, it would be good if you understand just somewhat intuitively what ConvNets do each layer before coding

In [8]:
#define our convnet model
def ConvNet(inputs):
    #Remember, our conv layers take inputs of size [batch_size, width, height, channels]. Here's an example of one layer
    first_layer = tf.layers.conv2d(inputs=inputs, filters=32, kernel_size=(3, 3), padding='valid', activation=tf.nn.relu)
    """ Let's walk through what we have defined. We take in the inputs and are creating an output of
    of [batch_size, height-1, width-1, filters]. Each filter is a product of scanning the kernel of size [3, 3]
    to our original image (In this case the height and width don't change because we made padding='same' for ease. 
    Normally padding is set to 'valid' which reduces image size corresponding to kernel size. You should look into
    this to understand for the future)
    TODO: Apply one more conv layers with half the current filters and an output layer for classification
    NOTE: Our output layer should be size [batch_size, 10] which can't be made by a conv layer which ouputs a 4D tensor.
    What can we do to fix this? (Message me if you need a hint :))
    """
    first_layer = tf.layers.conv2d(inputs=inputs, filters=16, kernel_size=(3, 3), padding='valid', activation=tf.nn.relu)
    #first_layer = tf.layers.max_pooling2d(first_layer, pool_size=7, strides=2)
    first_layer = tf.layers.flatten(first_layer)
    output_layer = tf.layers.dense(first_layer, units=10, activation=tf.nn.softmax)
    
    return output_layer


In [9]:
#Now let's walk through the same model building as last time
inputs=tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
#We define our input differently in this case

labels = tf.placeholder(tf.float32, shape=[None, 10])

outputs = ConvNet(inputs)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=labels))

trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

_, accuracy = tf.metrics.accuracy(tf.argmax(labels, 1), tf.argmax(outputs, 1))

#Let's set some hyperparamaters for batch size and epochs
batch_size = 128
num_epochs = 10
    

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
init=tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
with tf.Session() as sess:
    #each call to sess.run() runs the approriate input within our tf session
    sess.run(init)
    sess.run(local_init)
    print("Starting to run session...")
    for epoch in range(num_epochs): #iterate through each epoch
        for i in range(x_train.shape[0]//batch_size): #iterate through the number of batches we can make from our samples
            batch_images = x_train[i*batch_size:(i+1)*batch_size]
            batch_labels = y_train[i*batch_size:(i+1)*batch_size]
            #We have a batch of training images and labels. 
            #Now we need to run the model by inputing these into the place holders we made
            _ = sess.run([trainer], feed_dict={inputs: batch_images, labels: batch_labels})
        #Checking our accuracy each epoch...
        size = x_test.shape[0]//num_epochs
        test_images = x_test[epoch*size:(i+1)*size]
        test_labels = y_test[epoch*size:(i+1)*size]
        test_accuracy = sess.run([accuracy], feed_dict = {inputs: test_images, labels: test_labels})
        print("Accuracy at epoch {} is {}".format(epoch, test_accuracy))

Starting to run session...
Accuracy at epoch 0 is [0.9506]
Accuracy at epoch 1 is [0.95673686]
Accuracy at epoch 2 is [0.9611111]
Accuracy at epoch 3 is [0.96402943]
Accuracy at epoch 4 is [0.966625]
Accuracy at epoch 5 is [0.9682222]
Accuracy at epoch 6 is [0.9693061]
Accuracy at epoch 7 is [0.96982694]
Accuracy at epoch 8 is [0.9703148]
Accuracy at epoch 9 is [0.9702727]
